In [19]:
import pandas as pd
from src.connections.connections import DatabaseConnection
database_connection = DatabaseConnection()
name = "2023/03/28-21:03"
with database_connection.connect() as connect:
    df = pd.read_sql_table(name+"_flux", connect)
    df_iter = pd.read_sql_table(name+"_iter",connect)
    df = check_if_destination_doesnt_exist(df)
    avg_nation = df_iter.copy()
    avg_nation["State_Wealth"] = avg_nation["Avg Wealth"]*avg_nation["People"]
    average_nation_wealth = avg_nation.groupby('Iteration').agg({"State_Wealth": "sum", "People":"sum"}).reset_index()
    average_nation_wealth['Nation Avg Wealth'] = average_nation_wealth['State_Wealth']/average_nation_wealth["People"]
    df_destination = df.merge(df_iter[["State","Iteration","Avg Wealth", "Max Wealth", "Std Dev Wealth", "Min Wealth", "People"]].rename(columns={"Avg Wealth": "Previous Avg Wealth", "Max Wealth": "Previous Max Wealth", "Std Dev Wealth": "Previous Std Dev Wealth", "Min Wealth": "Previous Min Wealth", "People": "Previous People"}), how = "left", left_on=["destination", "iteration"], right_on=["State","Iteration"])
    df_destination = df_destination.merge(df_iter[["State","Iteration","Avg Wealth", "Max Wealth", "Std Dev Wealth", "Min Wealth", "People"]].rename(columns={"Avg Wealth": "Avg Entries State Wealth", " Wealth": "Max Entries State Wealth", "Std Dev Wealth": "Std Dev Entries State Wealth", "Min Wealth": "Min Entries State Wealth", "People": "Entries State People"}), how = "left", left_on=["origin", "iteration"], right_on=["State","Iteration"])
    df_destination = df_destination.groupby(["destination","iteration"]).agg(
        {"count":"sum", 
         "Previous Avg Wealth": "mean", "Previous Max Wealth": "mean", "Previous People": "mean", 
         "Avg Entries State Wealth": "mean", "Min Entries State Wealth": "mean",
         "wealth_mean": "mean", "wealth_std":"mean", "wealth_max": "max", "wealth_min":"min"
         }).reset_index().rename(columns={"count":"entries"})
    
    df_destination = df_destination[
        ["destination","iteration", "entries",
         "wealth_mean", "wealth_std", "wealth_max", "wealth_min",
          "Previous Avg Wealth", "Previous Max Wealth", "Previous People", 
          "Avg Entries State Wealth", "Min Entries State Wealth"]
        ]

    df_origin = df.copy()
    df_origin["count"] = -df_origin["count"]
    df_origin = df_origin.groupby(["origin","iteration"]).agg({"count":"sum", "wealth_mean":"mean"}).reset_index().rename(columns={"count":"departures", "wealth_mean": "departures_wealth_mean"})[["origin","departures", "iteration", "departures_wealth_mean"]]
    df_final = df_destination.merge(df_origin, how = "left", left_on = ["destination", "iteration"], right_on=["origin", "iteration"])
    
    df_final = df_final[
        ["destination", "iteration","entries",
        "wealth_mean", "wealth_std", "wealth_max", "wealth_min",
        "Previous Avg Wealth", "Previous Max Wealth", "Previous People",
        "Avg Entries State Wealth", "Min Entries State Wealth",
        "departures", "departures_wealth_mean"]]
    df_final["delta"] = df_final["entries"] + df_final["departures"]
    df_final["State Wealth"] = df_final["Previous Avg Wealth"]*df_final["Previous People"]
    df_final["change"] = df_final["delta"]/df_final["Previous People"]
    df_final["Year"] = (df_final["iteration"]+1)/52
    df_final["total_flux"] = df_final["entries"]/6000
    df_final = df_final.merge(average_nation_wealth[['Iteration','Nation Avg Wealth']], how="left", left_on="iteration",right_on="Iteration")
    df_final.to_sql(name+"_analysis", connect, if_exists="replace", index=False)

C:\Users\ArthurDornelles\AppData\Local\Temp\ipykernel_36484\2951229353.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lines)


In [2]:
def check_if_destination_doesnt_exist(df:pd.DataFrame)->pd.DataFrame:
    df_destination = df[["destination","iteration"]].groupby(["destination", "iteration"]).sum().reset_index()
    df_origin = df[["origin","iteration"]].groupby(["origin", "iteration"]).sum().reset_index()
    lines = df_origin.merge(df_destination, left_on = ["origin","iteration"], right_on = ["destination","iteration"],how = "left", indicator=True)
    lines = lines[lines["_merge"]=="left_only"][["iteration","origin"]].rename(columns={"origin":"destination"})
    lines["count"] = 0
    df = df.append(lines)
    return df

df_teste=check_if_destination_doesnt_exist(df)
df_teste[df_teste["iteration"]==1923]

C:\Users\ArthurDornelles\AppData\Local\Temp\ipykernel_36484\2951229353.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lines)


,origin,destination,iteration,wealth_mean,count,wealth_min,wealth_std,wealth_max
576,1.0,2.0,1923,172.741971,555,116.356784,48.471402,350.653721
577,1.0,3.0,1923,155.066519,278,112.806739,37.210162,329.734274
578,2.0,1.0,1923,169.271707,764,118.287380,43.921746,356.740439
579,2.0,3.0,1923,182.108181,842,121.125444,44.612094,453.395275
580,3.0,1.0,1923,162.775398,311,112.740762,38.097073,342.033468
581,3.0,2.0,1923,182.804828,634,118.777470,44.244482,469.609114
582,3.0,5.0,1923,600.725458,5,502.372913,98.597439,749.399910
